In [6]:
USER = 'thomas'

In [7]:
from _recommend_helper import load_rated, dataset
import pandas as pd

In [8]:
try:
    pref = load_rated(USER)
except FileNotFoundError:
    raise FileNotFoundError(f'no preference file for {USER!r} found, make sure you ran ../rate_movies/rate_movies.ipynb')

In [9]:
ratings = dataset.load('ratings.csv').drop('timestamp', axis=1)
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [15]:
df_for_dataset = pd.concat([ratings, pref]).reset_index(drop=True)
df_for_dataset

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100888,thomas,8368,4.5
100889,thomas,5378,4.0
100890,thomas,2617,4.0
100891,thomas,2791,5.0
